<a id='intro'></a>
# Создание дашборда в Tableau

## Задание и инструкция по выполнению проекта:

Имеется потребность автоматизировать анализ пользовательского взаимодействия с карточками статей Яндекс.Дзена - собрать интересующие данные в дашборд. Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан дата-инженерами.<br>

**ИНСТРУКЦИЯ К ВЫПОЛНЕНИЮ ПРОЕКТА**<br>
**Шаг №1. Разработать ТЗ на дашборд**<br>
**Шаг №2. Создать дашборд**<br>
1. Скачать файл https://storage.yandexcloud.net/cloud-certs/CA.pem в папку с файлом .ipynb, в котором будем работать с базой;<br>
2. Создать коннекцию к базе;<br>
3. Выполнить sql-запрос из базы;<br>
4. В Юпитере написать код для выгрузки данных из таблицы dash_visits в файл dash_visits.csv;<br>
5. В Tableau Public на основе файла dash_visits.csv сформировать дашборд в соответствии с макетом;<br>
6. Опубликовать дашборд на сайте Tableau Public. Дашборд должен быть доступен для всего интернета (открывается в нескольких браузерах);<br>
7. Ответить на вопросы, используя дашборд:<br>
o	Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?<br>
o	Как много карточек генерируют источники с разными темами?<br>
o	Как соотносятся темы карточек и темы источников?<br>
**Шаг №3. Представить результаты в виде PDF-файла**<br> 
Подготовить небольшую презентацию с отчётом — ответы на вопросы и графики.

**ОПИСАНИЕ ДАННЫХ**<br>
Сырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw):<br>
•	Карточка отображена для пользователя (show);
•	Пользователь кликнул на карточку (click);
•	Пользователь просмотрел статью карточки (view).


## ТЗ на дашборд

[В начало](#intro)

•	Состав данных для дашборда:<br> 
o	История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);<br>
o	Разбивка событий по темам источников;<br>
o	Таблица соответствия тем источников темам карточек;<br>
•	По каким параметрам данные должны группироваться:<br> 
o	Дата и время;<br>
o	Тема карточки;<br>
o	Тема источника;<br>
o	Возрастная группа;<br>
•	Характер данных:<br>
o	История событий по темам карточек — абсолютные величины с разбивкой по минутам;<br>
o	Разбивка событий по темам источников — относительные величины (% событий);<br>
o	Соответствия тем источников темам карточек - абсолютные величины;<br>
•	Важность: все графики имеют равную важность;<br>
•	Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);<br>
•	База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;<br>
•	Частота обновления данных: один раз в сутки, в полночь по UTC.<br>

## Создание дашборда

[В начало](#intro)

In [1]:
# Импортируем библиотеки Питона:
import pandas as pd #
from sqlalchemy import create_engine #
from datetime import datetime, time #
!pip install psycopg2 #
import warnings #

# Изменяем настройки отображения обработки кода в Пандасе, чтобы предупрежедения не выводились:
warnings.filterwarnings("ignore")
# Изменяем настройки отображения столбцов в Пандасе, чтобы выводились все столбцы:
pd.set_option('display.max_columns', None)
# Изменяем настройки отображения столбцов в Пандасе, чтобы выводились 200 символов в ячейке:
pd.options.display.max_colwidth = 200


# Создаем подключение к базе данных:
# Задаём параметры подключения к БД:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', # компьютер с базой данных
             'port': 6432, # порт подключения к компьютеру с базой данных
             'db': 'data-analyst-zen-project-db'} # название базы данных
# Формируем строку соединения с БД:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# Подключаемся к БД:
engine = create_engine(connection_string) 

# Формируем sql-запрос:
# в задании была таблица log_raw, а не dash_visits, 
# но преподаватель Ольга Макаревич говорит что это осталось от предыдущей версии спринта
query = ''' SELECT *
              FROM dash_visits
        '''

# Выполняем запрос и сохраняем результат выполнения в DataFrame.
# Sqlalchemy автоматически установит названия колонок такими же, как у таблицы в БД. 
# Указываем индексную колонку с помощью index_col.
dash_visits = pd.io.sql.read_sql(query, con = engine)

# Контроль итоговой таблицы:
display(dash_visits.head(5))
dash_visits.info()

# Выгружаем данные из таблицы dash_visits в файл dash_visits.csv:
dash_visits.to_csv('dash_visits.csv')

# Дальнейшее формирование дашборда осуществляем в Tableau.

ERROR: Invalid requirement: '#'


,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


Дальнейшее формирование дашборда осуществляем в Tableau.

## Общий вывод
[В начало](#intro)

Ссылка на дашборд:<br>
<br>
По результаатм анализа данных с помощью дашборда возможно дать следующие ответы на вопросы:<br>

1) Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?<br>
   - Пиковое значение в районе 60 тысяч взаимодействий.
   
2) Как много карточек генерируют источники с разными темами?<br>
   - Минимальное значение (2698 взаимодействий или 0,85% от общего количества) - генерирует тема "Финансы", а максимальное занчение (33309 взаимодействий или 10,74%) - "Семейные отношения".
   
3) Как соотносятся темы карточек и темы источников?<br>
   - Возможно установить неожиданные взаимосвязи между темами источников и карточек. Так, например, источник "Технологии" + карточка "Рассказы" дает только 1 взаимодействие. Лидирует пара источник "Путешествия" + карточка "Рассказы" (4587 взаимодействий).
<br>

Ссылка на дашборд: <https://public.tableau.com/app/profile/yakov8200/viz/Dash_03-01/Dash_1?publish=yes> 

Ссылка на презентацию в формате PDF: <https://disk.yandex.ru/i/b8eiYjDQq-A0cA> 